In [28]:
from langgraph.graph import END, START, StateGraph
from langgraph.types import Send,interrupt, Command
from typing import TypedDict
import subprocess
from openai import OpenAI
import textwrap
from langchain.chat_models import init_chat_model
from typing_extensions import Annotated
import operator
import base64
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

llm = init_chat_model(model="gpt-4o-mini")

class State(TypedDict):

    video_file: str
    audio_file: str
    transcription: str
    summaries: Annotated[list[str], operator.add]
    thumbnail_prompts: Annotated[list[str], operator.add]
    thumbnail_sketches: Annotated[list[str], operator.add]
    final_summary: str
    user_feedback: str
    chosen_prompt: str

In [29]:
from openai.resources.chat import CompletionsWithStreamingResponse


def extract_audio(state: State):
    output_file = state["video_file"].replace("mp4","mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        "-y",
        output_file,
    ]
    subprocess.run(command)
    return {"audio_file": output_file}

def transcribe_audio(state: State):
    client = OpenAI()
    with open(state["audio_file"], "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            language="ko",
            prompt="Jacqueline, Isabele"
        )
    return {"transcription": transcript}

def dispatch_summarizers(state: State):
    transcription = state["transcription"]
    chunks = []
    for i, chunk in enumerate(textwrap.wrap(transcription, 100)):
        chunks.append({"id": i+1, "chunk": chunk})
    return [Send("summarize_chunk", chunk) for chunk in chunks]
def summarize_chunk(chunk:str):
    chunk_id = chunk["id"]
    chunk = chunk["chunk"]

    response = llm.invoke(
        f"""Please summarize the following text:

        Text: {chunk}
        """
    )
    summary = f"[Chunk {chunk_id}] {response.content}"
    return {
        "summaries": [summary],
        }

def mega_summary(state: State):
    all_summaries = "\n".join(state["summaries"])
    prompt = f"""
    You are given multiple summaries of different chunks from a video transcription.
    
    Please create a comprehensive final summary that combines all the key points.

    Individual summaries:

    {all_summaries}
    """        

    response = llm.invoke(prompt)

    return {
        "final_summary": response.content
    }

def dispatch_artists(state: State):
    return [
        Send("generate_thumbnails", {"id": i, "summary": state["final_summary"]}) 
        for i in [1,2,3,4,5]
    ]

def generate_thumbnails(args):
    concept_id = args["id"]
    summary = args["summary"]

    prompt = f"""
    Based on this video summary, create a detailed visual prompt for a YouTube thumbnail.

    Create a detailed prompt for generationg a thumbnail image that would attract viewers. Include:
    - Main visual elements
    - Color scheme
    - Text overlay suggestions
    - Overall composition 

    Summary: {summary}
    """ 

    response = llm.invoke(prompt)

    thumbnail_prompt = response.content

    client = OpenAI()

    result = client.images.generate(
        model="gpt-image-1",
        prompt=thumbnail_prompt,
        quality="low",
        moderation="low",
        size="auto",
    )

    image_bytes = base64.b64decode(result.data[0].b64_json)
    filename = f"thumbnail_{concept_id}.jpeg"

    with open(filename, "wb") as file:
        file.write(image_bytes)
    
    return {
        "thumbnail_prompts": [thumbnail_prompt],
        "thumbnail_sketches": [filename],
    }

def human_feedback(state: State):
    answer = interrupt({
        "chosen_thumbnail": "which thumbnail do you like the most?",
        "feedback": "Provide any feedback or changes you'd like to make for the thumbnails"
    })

    user_feedback = answer["user_feedback"]
    chosen_prompt = answer["chosen_prompt"]

    return {
        "user_feedback": user_feedback,
        "chosen_prompt": state["thumbnail_prompts"][chosen_prompt-1],
    }

def generate_hd_thumbnail(state: State):
    
    chosen_prompt = state["chosen_prompt"]
    user_feedback = state["user_feedback"]

    prompt = prompt = f"""
    You are a professional YouTube thumbnail designer. Take this original thumbnail prompt and create an enhanced version that incorporates the user's specific feedback.

    ORIGINAL PROMPT:
    {chosen_prompt}

    USER FEEDBACK TO INCORPORATE:
    {user_feedback}

    Create an enhanced prompt that:
        1. Maintains the core concept from the original prompt
        2. Specifically addresses and implements the user's feedback requests
        3. Adds professional YouTube thumbnail specifications:
            - High contrast and bold visual elements
            - Clear focal points that draw the eye
            - Professional lighting and composition
            - Optimal text placement and readability with generous padding from edges
            - Colors that pop and grab attention
            - Elements that work well at small thumbnail sizes
            - IMPORTANT: Always ensure adequate white space/padding between any text and the image borders
    """

    response = llm.invoke(prompt)

    final_thumbnail_prompt = response.content

    client = OpenAI()

    result = client.images.generate(
        model="gpt-image-1",
        prompt=final_thumbnail_prompt,
        quality="high",
        moderation="low",
        size="auto",
    )

    image_bytes = base64.b64decode(result.data[0].b64_json)

    with open("thumbnail_final.jpg", "wb") as file:
        file.write(image_bytes)
    

In [30]:
graph_builder = StateGraph(State)

graph_builder.add_node("extract_audio", extract_audio)
graph_builder.add_node("transcribe_audio", transcribe_audio)
graph_builder.add_node("summarize_chunk", summarize_chunk)
graph_builder.add_node("mega_summary", mega_summary)
graph_builder.add_node("generate_thumbnails", generate_thumbnails)
graph_builder.add_node("human_feedback", human_feedback)
graph_builder.add_node("generate_hd_thumbnail", generate_hd_thumbnail)

graph_builder.add_edge(START, "extract_audio")
graph_builder.add_edge("extract_audio", "transcribe_audio")
graph_builder.add_conditional_edges(
    "transcribe_audio", dispatch_summarizers, ["summarize_chunk"]
)
graph_builder.add_edge("summarize_chunk", "mega_summary")
graph_builder.add_conditional_edges(
    "mega_summary", dispatch_artists, ["generate_thumbnails"]
)
graph_builder.add_edge("generate_thumbnails", "human_feedback")
graph_builder.add_edge("human_feedback", "generate_hd_thumbnail")
graph_builder.add_edge("generate_hd_thumbnail", END)

graph = graph_builder.compile(checkpointer=memory)

In [31]:
config = {
    "configurable":{
        "thread_id":1,
    },
}

In [32]:
graph.invoke(
    {"video_file": "fun.mp4"},
    config=config,
    )

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.6.3.2)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gpl --enable-libsvtav1 --enable-libopus --enable-libx264 --enable-libmp3lame --enable-libdav1d --enable-libvpx --enable-libx265 --enable-openssl --enable-videotoolbox --enable-audiotoolbox --enable-neon
  libavutil      60.  8.100 / 60.  8.100
  libavcodec     62. 11.100 / 62. 11.100
  libavformat    62.  3.100 / 62.  3.100
  libavdevice    62.  1.100 / 62.  1.100
  libavfilter    11.  4.100 / 11.  4.100
  libswscale      9.  1.100 /  9.  1.100
  libswresample   6.  1.100 /  6.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'fun.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    encoder         : Google
  Duration: 00:02:41.49, sta

{'video_file': 'fun.mp4',
 'audio_file': 'fun.mp3',
 'transcription': '자베리는 지금 뉴스 촬영에 푹 빠져있다. 무화지경에 빠져 챌린지 중인 자베리. 보고 있으면 피가 거꾸로 솟는다. 어머나! 나가요! 피가 다 엉췄... 자베라! 정례치대 진짜. 쟤 드라이밍이 크레이지! 또끝내야지 정말! 한편, 귤락을 골라내는 섬세합니다. 영국의 그레이스도 K-드라마에 푹 빠졌다. 아, 정말 이거 다 빨라, 정말로! 진짜 스크라이스다, 자베라 정말로 진짜! 어디론가 전화를 거는 그레이스. 여보, 오늘 올 때 마이크랑 그러면 사와. 당연히 레드지, 내가 피운 거! 몰라, 오늘은 그냥 시그널 피워야겠다, 오늘 정말로 진짜! 오늘은 피워야 돼! 그냥 사와! 이날은 담배 없이는 버틸 수 없었다. 끊어요! 거의 영국인답게 차는 잊지 않는다. 그래, 이제는 나도 어쩔 수가 없어. 그냥 가! 그래! 불현듯 어디론가 전화를 거는 그레이스. 노래를 이은미 창법으로 부른다. 여러분, 누구세요? 어, 나야 유난다! 나 그레이스야! 어, 그레이스! 나 지금 오피스에서 스테인리스에서 있었는데 유난다! 유난다! 그때 에어 삥삥가 그거 무슨 하숙지 같은 거 끝나고 그랬었지? 어, 맞아요, 언니! 우리 마이크를 비거리 힐 이제 우리 집 팔고 한국 가서 그거는 좀 하고 그랬거든? 어머, 그래? 소문에 린나 언니가 막 한국에서 브랜드처럼 산다고 막 그랬는데. 이제 린나 아니고 제클린이라니까! 이제 잘 있으니까 이상한 소리 좀 그만하고 다녀, 너! 제클린 험담에 속상해진 그레이스. 쉿! 그레이스야! 뭐야? 다시 시작된 유난다의 이은미 창법. 어디론가 또다시 전화를 거는 그레이스.\n',
 'summaries': ['[Chunk 1] 자베리는 현재 뉴스 촬영에 몰두하고 있으며, 그 모습이 나를 당황스럽게 한다. 챌린지에 참여하고 있는 자베리를 보고 있으면 감정이 복잡해진다.',
  '[Chunk 2] The text expresses excitement a

In [33]:
response = {
    "user_feedback": "Make sure the fellas are smiling, the lady looks Korean, give it a photo realistic, 3d style.",
    "chosen_prompt": 2,
}

graph.invoke(
    Command(resume=response),
    config=config,
)

KeyboardInterrupt: 